In [5]:
import pandas as pd
import distances
from Bio.PDB import *
import os

In [6]:
# Dataset with all the PDB structures for all kinases (123 out of 280)
uniprot_pdb = pd.read_csv('../datasets/uniprot_pdb.tsv', sep= '\t')
# Mapping all pdbs for the proteins
uniprot_pdb = uniprot_pdb[['uniprot_acc', 'pdb']].drop_duplicates()
uniprot_pdb.rename(columns= {'uniprot_acc': 'uniprot'}, inplace= True)
uniprot_pdb

,uniprot,pdb
0,O08722,3g5b
1,O08808,4uwx
2,O08808,1z2c
3,O08808,3eg5
4,O08808,2bnx
...,...,...
2481,Q9Y613,6xf1
2482,Q9Y613,3dad
2483,Q9Y613,6xf2
2484,Q9Z0R4,3jv3


In [18]:
# proteins with structure in PDB
uniprot_pdb.uniprot.nunique()

123

In [9]:
uniprot_pdb.pdb.nunique()

2470

In [10]:
# Our kinases dataset (280 proteins)
kinases = pd.read_csv('../datasets/kinases.tsv', sep= '\t')
kinases.columns = kinases.columns.str.lower().str.replace(" ", "_")
kinases.rename(columns= {'kd_(start)': 'kd_start', 'kd_(end)': 'kd_end'}, inplace= True)

In [17]:
# unique proteins
kinases.uniprot.nunique()

280

In [44]:
# list with unique kinases uniprot accessions
# kl = kinases.uniprot.unique().tolist()
# with open('../datasets/uniprot_acc_kinases.txt', 'w', encoding= 'utf-8') as fp:
#     for uniprot in kl:
#         # write each accession on a new line
#         fp.write("%s\n" % uniprot)

In [11]:
kd = kinases[['uniprot', 'kd_start', 'kd_end']]

In [9]:
# # Table with pdb ids of kinases containing CREs dataset
# pdbs_kd_cre = pd.read_csv('../datasets/pdbs_kd_cre.tsv', sep= '\t')
# pdbs_kd_cre

In [10]:
#pdbs_kd_cre.uniprot_acc.nunique() # 42 proteins with structure that includes KD and CRE

In [12]:
# Catalitic sites brought from uniprot
cat_sites = pd.read_csv('../datasets/cat_sites_uniprot.tsv', sep= '\t')
cat_sites.rename(columns= {'pos': 'cs'}, inplace= True)
# mapear el sitio catalitico al rango de la kinasa
cat_sites = cat_sites.merge(kd)
cat_sites

,uniprot,cs,kd_start,kd_end
0,A0A2I0BVG8,191,56,325
1,A0A509AFG4,230,110,365
2,A0A509AHB6,190,57,324
3,A0A509AKL0,668,545,802
4,A0A509AQE6,238,118,372
...,...,...,...,...
305,Q9ZSA4,156,28,290
306,Q9ZUZ2,271,143,405
307,Q9ZV15,258,134,392
308,W0LYS5,153,31,287


In [13]:
# Control: CS is within KD?
cat_sites['ctrl'] = False
for i in cat_sites.index:
    rg = range(cat_sites.kd_start[i], cat_sites.kd_end[i])
    if cat_sites.cs[i] in rg:
        cat_sites['ctrl'][i] = True

/home/alvaro/miniconda3/envs/alvenv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
cat_sites.ctrl.sum()

298

In [15]:
cat_sites = cat_sites[cat_sites.ctrl == True]

In [16]:
cat_sites.uniprot.nunique()

278

In [19]:
# Map catalytic sites to the proteins
uniprot_pdb_cat = uniprot_pdb.merge(cat_sites[['uniprot', 'cs']], how= 'left')
uniprot_pdb_cat

,uniprot,pdb,cs
0,O08722,3g5b,NaN
1,O08808,4uwx,NaN
2,O08808,1z2c,NaN
3,O08808,3eg5,NaN
4,O08808,2bnx,NaN
...,...,...,...
2722,Q9Y613,6xf1,NaN
2723,Q9Y613,3dad,NaN
2724,Q9Y613,6xf2,NaN
2725,Q9Z0R4,3jv3,NaN


---

## Measure interatomic distances
use all heavy atoms (all_atoms_selector)

### Download PDBs files

In [8]:
#parser = PDBParser()

In [17]:
def download_pdb(pdbs, pdir, file_format= 'pdb'):
    """
    download a pdb file from web using PDB package from biopython
    pdbs: list of PDBs accesion
    pdir: output folder
    file_format: format to download PDB structure
    """
    from Bio.PDB import PDBList
    #parser = PDBParser()
    pdbl = PDBList()
    for pdb in pdbs:
        pdbl.retrieve_pdb_file(pdb, file_format= 'pdb', pdir= pdir)

In [12]:
pdbs = uniprot_pdb.pdb.unique().tolist()

In [18]:
download_pdb(l, '../raw_data/')

KeyboardInterrupt: 

In [19]:
# # download structures
# pdbl = PDBList()
# for pdb in pdbs:
#     pdbl.retrieve_pdb_file(pdb, file_format= 'pdb', pdir= '../raw_data/pdb_files')

In [7]:
# cter = pd.read_csv('../raw_data/uniprot_kd_cre_cter.gz', sep= '\t', compression= 'gzip')
# cter.columns = cter.columns.str.lower().str.replace(" ", "_")
# cter.drop(columns= ['from', 'entry_name'], inplace= True)

In [8]:
pdbs_kd_cre[pdbs_kd_cre.uniprot_acc.isin(cter.entry)].uniprot_acc.nunique()

22

### Measure distances

In [1]:
# use all_atoms selector and then filter for those distances below 6 A
# from distances import all_atoms_selector
# dist_all_6yp3 = distances.calculate_distances(pdb_source= '../raw_data/6yp3.pdb', atom_selector= carbon_alfa_selector, include_extra_info= True)

con esto, solo faltaria agregar una col de catalytic site (bool)

In [4]:
def distances_carbon_alfa(path, uniprot_pdb):
    '''
    path: path containing PDB files
    uniprot_pdb: dataframe of mapping uniprot-pdb
    '''
    from distances import carbon_alfa_selector

    path = '../raw_data/pdb_files/'
    # iterate over the uniprot files
    for filename in os.listdir(path):
        
        #print(filename)
        f = os.path.join(path, filename)
        #print(f)
        if os.path.isfile(f):

            dist = distances.calculate_distances(pdb_source= f, atom_selector= carbon_alfa_selector, include_extra_info= True)
            filename = filename.strip("pdb.ent")
            #distances.save_distances(dist, outfile= "../datasets/distances/" + str(filename)  + ".pdb")

            df = pd.DataFrame(dist, columns = ['chain_a', 'pos_a', 'aa_a', 'atom_a', 'chain_b', 'pos_b', 'aa_b', 'atom_b', 'dist'])
            df = df[(df.pos_a > 0) & (df.pos_b > 0)]
            df = df[df.dist <= 6]

            uniprot = uniprot_pdb[uniprot_pdb.pdb == filename]
            if uniprot.size != 0:
                uniprot = uniprot.values[0][0]
                df["uniprot"] = uniprot
                df["pdb"] = filename
                print(f"writing file for {filename} pdb")
                df.to_csv('../datasets/distances/dataframes/' + uniprot + "_" + filename + '.csv', index= False)


In [7]:
# for all heavy atoms
def distances_all_atoms(path, uniprot_pdb, output, threshold = 6):
    '''
    Given a pdb file, calculates the interatomic distances among heavy atoms
    Returns: a data frame with distances < 6 A
    path: path containing PDB files
    uniprot_pdb: dataframe of mapping uniprot-pdb
    output: output path
    '''
    import os
    from distances import all_atoms_selector
    from Bio.PDB.PDBParser import PDBParser

    # iterate over the pdb files
    for filename in os.listdir(path):
        f = os.path.join(path, filename) # is the complete path to the pdb file

        if os.path.isfile(f):

            filename = filename.rstrip(".ent").lstrip("pdb") # get pdb accesion
            try:
                pdb = PDBParser().get_structure(str(filename), path + filename)
                uniprot = uniprot_pdb[uniprot_pdb.pdb == filename] 
                if uniprot.size != 0:
                    uniprot = uniprot.values[0][0] # get uniprot accesion
                    output_file = output + uniprot + "_" + filename + '.csv'
                
                # check if output already exists
                if not os.path.exists(output_file):

                    # Check if heteroatoms
                    for residue in pdb.get_residues():
                        tags = residue.get_full_id()
                        # tags contains a tuple with (Structure ID, Model ID, Chain ID, (Residue ID))
                        # Residue ID is a tuple with (*Hetero Field*, Residue ID, Insertion Code)

                        # Thus you're interested in the Hetero Field, that is empty if the residue
                        # is not a hetero atom or have some flag if it is (W for waters, H, etc.)

                        if tags[3][0] != " ":
                            # The residue is a heteroatom
                            print(f'residues are hetatom: {tags}')
                        else:
                            # It is not
                            print('residues are not hetatom')



                    # calculate distance
                    # param pdb_source: a path to a pdb file, a Bio.PDB.Structure or a
                    # Bio.PDB.Model
                    dist = distances.calculate_distances(
                        pdb_source= f, # cambiar por archivo q no contenga los hetatoms
                        atom_selector= all_atoms_selector,
                        include_extra_info= True
                    )
                    # df with distances filtered by threshold
                    df = pd.DataFrame(dist, columns = ['chain_a', 'pos_a', 'aa_a', 'atom_a', 'chain_b', 'pos_b', 'aa_b', 'atom_b', 'dist'])
                    df = df[(df.pos_a > 0) & (df.pos_b > 0)]
                    df = df[df.dist <= threshold]

                    df["uniprot"] = uniprot
                    df["pdb"] = filename
                    print(f"writing file for {filename} pdb")
                    df.to_csv(output_file, index= False)
            except:
                print(f'no uniprot found for {filename}')


In [11]:
def select_aa(Select):
    from constant import ALL_THREE_LETTER_AA
    """
    Select all atoms from protein residues.
    """
    def accept_atom(self, atom):
        return True
    def accept_chain(self, chain):
        return True
    def accept_residue(self, residue):
        return ALL_THREE_LETTER_AA.get(residue.resname.strip()) != None
    def accept_model(self, model):
        return True  

In [ ]:
# for all heavy atoms
def distances_all_atoms(path, uniprot_pdb, output, threshold = 6):
    '''
    Given a pdb file, calculates the interatomic distances among heavy atoms
    Returns: a data frame with distances < 6 A
    path: path containing PDB files
    uniprot_pdb: dataframe of mapping uniprot-pdb
    output: output path
    '''
    import os
    from distances import all_atoms_selector
    from Bio.PDB.PDBParser import PDBParser
    from Bio.PDB.PDBIO import PDBIO, Select
    import warnings
    from io import StringIO


    warnings.filterwarnings("ignore") 
    parser = PDBParser()
    string_io = StringIO()
    pdbio = PDBIO()

    # iterate over the pdb files
    for filename in os.listdir(path):
        f = os.path.join(path, filename) # is the complete path to the pdb file

        if os.path.isfile(f):

            pdb_structure = parser.get_structure(id="XXX1", file= f) # read raw pdb file
            pdbio.set_structure(pdb_structure)     
            pdbio.save(string_io, select= select_aa())
            string_io = StringIO(string_io.getvalue())
            pdb_structure = parser.get_structure("XXX1", string_io)

            filename = filename.rstrip(".ent").lstrip("pdb") # get pdb accesion

            try:
                #pdb = PDBParser().get_structure(str(filename), path + filename)
                uniprot = uniprot_pdb[uniprot_pdb.pdb == filename] 
                if uniprot.size != 0:
                    uniprot = uniprot.values[0][0] # get uniprot accesion
                    output_file = output + uniprot + "_" + filename + '.csv'
                
                # check if output already exists
                if not os.path.exists(output_file):

                    ################################################
                    # # Check if heteroatoms
                    # for residue in pdb_structure.get_residues():
                    #     tags = residue.get_full_id()
                    #     # tags contains a tuple with (Structure ID, Model ID, Chain ID, (Residue ID))
                    #     # Residue ID is a tuple with (*Hetero Field*, Residue ID, Insertion Code)

                    #     # Thus you're interested in the Hetero Field, that is empty if the residue
                    #     # is not a hetero atom or have some flag if it is (W for waters, H, etc.)

                    #     if tags[3][0] != " ":
                    #         # The residue is a heteroatom
                    #         print(f'residues are hetatom: {tags}')
                    #     else:
                    #         # It is not
                    #         print('residues are not hetatom')
                    ################################################

                    # QUEDE AQUI
                    # calculate distance
                    # param pdb_source: a path to a pdb file, a Bio.PDB.Structure or a
                    # Bio.PDB.Model
                    dist = distances.calculate_distances(
                        pdb_source= pdb_structure, # f, cambiar por archivo q no contenga los hetatoms
                        atom_selector= all_atoms_selector,
                        include_extra_info= True
                    )
                    # df with distances filtered by threshold
                    df = pd.DataFrame(dist, columns = ['chain_a', 'pos_a', 'aa_a', 'atom_a', 'chain_b', 'pos_b', 'aa_b', 'atom_b', 'dist'])
                    df = df[(df.pos_a > 0) & (df.pos_b > 0)]
                    df = df[df.dist <= threshold]

                    df["uniprot"] = uniprot
                    df["pdb"] = filename
                    print(f"writing file for {filename} pdb")
                    df.to_csv(output_file, index= False)
            except:
                print(f'no uniprot found for {filename}')

In [ ]:
import warnings
from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO, Select
from io import StringIO

warnings.filterwarnings("ignore") 
parser = PDBParser()
pdb_structure = parser.get_structure(id="XXX1", file= pdb_content) # raw pdb
string_io = StringIO()
pdbio = PDBIO()
pdbio.set_structure(pdb_structure)     
pdbio.save(string_io, select= select_aa())
string_io = StringIO(string_io.getvalue())
pdb_structure = parser.get_structure("XXX1", string_io)


In [6]:
#distances_all_atoms('../raw_data/pdb_files/', uniprot_pdb, output= '../datasets/distances/dataframes/')

In [10]:
def select_aa(Select):
    from constant import ALL_THREE_LETTER_AA
    """
    Select all atoms from protein residues.
    """
    def accept_atom(self, atom):
        return True
    def accept_chain(self, chain):
        return True
    def accept_residue(self, residue):
        return ALL_THREE_LETTER_AA.get(residue.resname.strip()) != None
    def accept_model(self, model):
        return True  


In [ ]:
import warnings
from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO, Select
from io import StringIO

warnings.filterwarnings("ignore") 
parser = PDBParser()
pdb_structure = parser.get_structure(id="XXX1", file= pdb_content) # raw pdb
string_io = StringIO()
pdbio = PDBIO()
pdbio.set_structure(pdb_structure)     
pdbio.save(string_io, select= select_aa())
string_io = StringIO(string_io.getvalue())
pdb_structure = parser.get_structure("XXX1", string_io)


In [9]:
# know if a residue is a heteroatom
from Bio.PDB.PDBParser import PDBParser
pdb = PDBParser().get_structure("3uyo", "../raw_data/pdb_files/pdb3uyo.ent")

for residue in pdb.get_residues():
    tags = residue.get_full_id()

    # tags contains a tuple with (Structure ID, Model ID, Chain ID, (Residue ID))
    # Residue ID is a tuple with (*Hetero Field*, Residue ID, Insertion Code)

    # Thus you're interested in the Hetero Field, that is empty if the residue
    # is not a hetero atom or have some flag if it is (W for waters, H, etc.)

    if tags[3][0] != " ":
        # The residue is a heteroatom
        print(f'residues are hetatom: {tags}')
    else:
        # It is not
        print(f'residues are not hetatom {tags}')

residues are not hetatom ('3uyo', 0, 'A', (' ', 134, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 135, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 136, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 137, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 138, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 139, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 140, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 141, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 142, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 143, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 144, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 145, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 146, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 147, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 148, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 149, ' '))
residues are not hetatom ('3uyo', 0, 'A', (' ', 150, ' '

/home/alvaro/miniconda3/envs/alvenv/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2028.
  PDBConstructionWarning,
/home/alvaro/miniconda3/envs/alvenv/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 2118.
  PDBConstructionWarning,


In [2]:
len("")

0

In [12]:
# Mapping SIFT positions in PDB and positions in UniProt
mapping = pd.read_csv('../raw_data/uniprot_segments_observed.tsv.gz', sep= '\t', compression= 'gzip', skiprows= 1)
mapping.columns = mapping.columns.str.lower().str.replace(" ", "_")
mapping = mapping[['pdb', 'chain', 'sp_primary', 'pdb_beg', 'pdb_end', 'sp_beg', 'sp_end']]
mapping

,pdb,chain,sp_primary,pdb_beg,pdb_end,sp_beg,sp_end
0,1wfc,A,Q16539,5,13,5,13
1,1wfc,A,Q16539,16,169,16,169
2,1wfc,A,Q16539,175,351,175,351
3,6aek,A,P06802,170,620,170,620
4,6aek,A,P06802,629,629,629,629
...,...,...,...,...,...,...,...
960081,7liw,B,P53396,2,139,2,139
960082,7liw,B,P53396,149,431,149,431
960083,7liw,B,P53396,487,1099,487,1099
960084,7liw,C,P53396,821,1099,821,1099


In [13]:
mapping[mapping.pdb == '6yp3']

,pdb,chain,sp_primary,pdb_beg,pdb_end,sp_beg,sp_end
12448,6yp3,A,P31947,1,231,1,231
12449,6yp3,P,Q04206,43,51,43,51


In [34]:
path = '../datasets/distances/dataframes/'
    # iterate over the distances files
for filename in os.listdir(path):
    
    print(filename)
    f = os.path.join(path, filename)
    #print(f)
    if os.path.isfile(f):
        print(f)
        l = filename.strip('.csv').split("_")
        uniprot, pdb = l[0], l[1]
        #pdb = l[1]

        #df = pd.read_csv(f)

        # Map for each df the catalytic sites

        # print(uniprot)
        # print(pdb)
        # print('----------')

        df = pd.read_csv(f).merge(cat_sites, how= 'left')
        df["cs"] = df.pos_a == df.cs
        if df.cs.sum().any():
            x = df[df.cs == True]
        x

O08967_1fgy.csv
../datasets/distances/dataframes/O08967_1fgy.csv
O08967
1fgy
----------
O08967_1fgz.csv
../datasets/distances/dataframes/O08967_1fgz.csv
O08967
1fgz
----------
O08967_1fhw.csv
../datasets/distances/dataframes/O08967_1fhw.csv
O08967
1fhw
----------
O08967_1fhx.csv
../datasets/distances/dataframes/O08967_1fhx.csv
O08967
1fhx
----------
O14745_1g9o.csv
../datasets/distances/dataframes/O14745_1g9o.csv
O14745
1g9o
----------
O14745_1gq4.csv
../datasets/distances/dataframes/O14745_1gq4.csv
O14745
1gq4
----------
O14745_1gq5.csv
../datasets/distances/dataframes/O14745_1gq5.csv
O14745
1gq5
----------
O14745_1i92.csv
../datasets/distances/dataframes/O14745_1i92.csv
O14745
1i92
----------
O14745_1sgh.csv
../datasets/distances/dataframes/O14745_1sgh.csv
O14745
1sgh
----------
P00519_1ab2.csv
../datasets/distances/dataframes/P00519_1ab2.csv
P00519
1ab2
----------
P00519_1awo.csv
../datasets/distances/dataframes/P00519_1awo.csv
P00519
1awo
----------
P00519_1bbz.csv
../datasets/dist

In [14]:
prueba = pd.read_csv('../datasets/distances/dataframes/P00523_1nlo.csv').merge(cat_sites, how= 'left')
prueba["pos"] = prueba.pos_a == prueba.pos
prueba

,chain_a,pos_a,aa_a,atom_a,chain_b,pos_b,aa_b,atom_b,dist,uniprot,pdb,pos
0,C,9,THR,CA,C,10,PHE,CA,3.823854,P00523,1nlo,False
1,C,9,THR,CA,C,32,LEU,CA,5.953827,P00523,1nlo,False
2,C,9,THR,CA,C,33,GLN,CA,5.008291,P00523,1nlo,False
3,C,10,PHE,CA,C,11,VAL,CA,3.828712,P00523,1nlo,False
4,C,10,PHE,CA,C,31,ARG,CA,5.792127,P00523,1nlo,False
...,...,...,...,...,...,...,...,...,...,...,...,...
140,N,75,PRO,CA,N,76,LEU,CA,3.823504,P00523,1nlo,False
141,N,76,LEU,CA,N,77,PRO,CA,3.808258,P00523,1nlo,False
142,N,77,PRO,CA,N,78,PRO,CA,3.808038,P00523,1nlo,False
143,N,78,PRO,CA,N,79,LEU,CA,3.817964,P00523,1nlo,False


In [29]:
if prueba.cs.sum().any():
    x = prueba[prueba.cs == True]
x

NameError: name 'x' is not defined

In [32]:
prueba2 = pd.read_csv('../datasets/distances/dataframes/P00519_1opl.csv').merge(cat_sites, how= 'left')
prueba2["cs"] = prueba2.pos_a == prueba2.cs
prueba2

,chain_a,pos_a,aa_a,atom_a,chain_b,pos_b,aa_b,atom_b,dist,uniprot,pdb,cs
0,A,81,ASP,CA,A,82,PRO,CA,3.847206,P00519,1opl,False
1,A,81,ASP,CA,A,84,LEU,CA,5.845485,P00519,1opl,False
2,A,82,PRO,CA,A,83,ASN,CA,3.839307,P00519,1opl,False
3,A,82,PRO,CA,A,84,LEU,CA,5.815568,P00519,1opl,False
4,A,83,ASN,CA,A,84,LEU,CA,3.899549,P00519,1opl,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2105,B,515,MET,CA,B,517,GLN,CA,5.641351,P00519,1opl,False
2106,B,515,MET,CA,B,518,GLU,CA,5.054327,P00519,1opl,False
2107,B,516,PHE,CA,B,517,GLN,CA,3.801774,P00519,1opl,False
2108,B,516,PHE,CA,B,518,GLU,CA,5.439814,P00519,1opl,False


In [33]:
if prueba2.cs.sum().any():
    x = prueba2[prueba2.cs == True]
x

,chain_a,pos_a,aa_a,atom_a,chain_b,pos_b,aa_b,atom_b,dist,uniprot,pdb,cs
728,A,363,ALA,CA,A,364,THR,CA,3.794177,P00519,1opl,True
729,A,363,ALA,CA,A,365,GLN,CA,5.481358,P00519,1opl,True
730,A,363,ALA,CA,A,366,ILE,CA,5.188070,P00519,1opl,True
1717,B,363,ALA,CA,B,364,THR,CA,3.794536,P00519,1opl,True
1718,B,363,ALA,CA,B,365,GLN,CA,5.482057,P00519,1opl,True
1719,B,363,ALA,CA,B,366,ILE,CA,5.188113,P00519,1opl,True


In [25]:
from itertools import product

In [31]:
list(product([1, 2], [3,4]))

[(1, 3), (1, 4), (2, 3), (2, 4)]

In [33]:
from distances import Distances

In [43]:
# for l in dist_cb_6yp3:
#     for elem in l:
#         print(elem)
#     # for chain_a, pos_a, chain_b, pos_b, dist in l:
#     #     print(chain_a, pos_a, chain_b, pos_b, dist)

In [35]:
Distances.is_contact(dist_cb_6yp3)

TypeError: is_contact() missing 4 required positional arguments: 'chain_a', 'pos_a', 'chain_b', and 'pos_b'

In [17]:
distances.save_distances(dist_6yp3, '../datasets/dist_6yp3')